#環境初始化，記得登入雲端硬碟

In [ ]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
# os.environ['PYSPARK_PYTHON'] = "/usr/bin/python"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')
file_dir = "/content/gdrive/My Drive/DataMining/DataWithPTT/"

!ls -l /content/gdrive/My\ Drive/DataMining/DataWithPTT

--2020-12-10 08:24:13--  https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.1, 2620:100:6017:1::a27d:201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6bnwn8u2hz19s59/init_env.sh [following]
--2020-12-10 08:24:13--  https://www.dropbox.com/s/raw/6bnwn8u2hz19s59/init_env.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4180b082795be2cb1b88119d41.dl.dropboxusercontent.com/cd/0/inline/BEznzdN9n3BuaHjtPfPRryUVZ6Wg05RKTPJPflM34Svl6mi7zoTXmzbhL50h9HK41TE013nI6OCwbWKJZi_rBEEK_C88HO25ueI4GQCg0ZSaTw/file# [following]
--2020-12-10 08:24:14--  https://uc4180b082795be2cb1b88119d41.dl.dropboxusercontent.com/cd/0/inline/BEznzdN9n3BuaHjtPfPRryUVZ6Wg05RKTPJPflM34Svl6mi7zoTXmzbhL50h9HK41TE013nI6OCwbWKJZi_rBEEK_C88HO25ueI4GQCg0ZSaTw/file
Resolving uc4180b08279

#以下工作假設:
1. 你的雲端硬碟路徑在/content/gdrive/My Drive/DataMining/DataWithPTT/，此路徑等於變數file_dir的內容
2. 你在該路徑底下有以下四個檔案

  IDTitle_content_string.txt

  new_PTT_IDTitle_content_string.txt

  word_entropy_table.pickle

  word_ID_table.pickle


In [ ]:
# 讀取word_entropy_table
import pickle

word_entropy_table = {}
with open(file_dir+'word_entropy_table.pickle', 'rb') as f:
  word_entropy_table = pickle.load(f)
f.close()

print(word_entropy_table['一個'])
print(word_entropy_table['關羽'])
print(word_entropy_table['劉備'])

1.3884529392843121
10.855945279099988
10.011451220629336


In [ ]:
# 讀取word_ID_table
import pickle

word_ID_table = {}
with open(file_dir+'word_ID_table.pickle', 'rb') as f:
  word_ID_table = pickle.load(f)
f.close()

print(word_ID_table['關羽'])

['1591025648', '1591518048', '1599705915', '1600838944', '1601638200', '1602683248', '2200', '2797', '2837', '2870', '8040', '8228', '12546', '12734', '15693', '16460', '16462', '17209', '17470', '21915', '22278', '22288', '22300', '22305', '22309', '22349', '23143', '31323', '33680', '33762', '49432', '50386', '50848', '51921', '54522', '55750', '55880', '59102', '71554', '75941', '78612', '79000', '82372', '83658', '87746', '91415', '93647', '110304', '114184', '119384', '133338', '134319', '140429', '140488', '140623', '142769', '150861', '154541', '173313', '175101', '179301', '187472', '187721', '190200', '193000', '201492', '201926', '212734', '232114', '260882', '267248', '267883', '272926', '275264', '278151', '285717', '308556', '368841', '390369', '390751', '390844', '392711', '422359', '449438', '451431', '451828', '459280', '464286', '464999', '471591', '480911', '504520', '509728', '509848', '526660', '534134', '539112', '541293', '566585', '568765', '571557', '609934', '6

In [ ]:
# 在VM上製作ID_word_count_table，會在ID_word_count_table_dir/底下
import pickle
import os

def splitByArticle(line):
  # 1 line is an article
  words = line.split()

  ID = words.pop(0)
  wordCountDict = {}
  for word in words:
    try:
      wordCountDict[word] += 1
    except:
      wordCountDict[word] = 1

  retTuple = (ID, wordCountDict)
  # retTuple格式: (ID, wordCountDict)

  return retTuple

def fileWriteBack(tp): 
  ID = tp[0]
  WCDict = tp[1]

  with open('ID_word_count_table_dir/'+ID+'.pickle', 'wb') as f:
    pickle.dump(WCDict, f)
  f.close()

os.makedirs(os.path.dirname('ID_word_count_table_dir/'), exist_ok=True)
ID_word_count_table_rdd = sc.textFile(file_dir+"IDTitle_content_string.txt")+sc.textFile(file_dir+"new_PTT_IDTitle_content_string.txt")
ID_word_count_table_rdd = ID_word_count_table_rdd.map(splitByArticle) \
                        .map(fileWriteBack)
ID_word_count_table_rdd.collect()

print('done')

!ls -l ID_word_count_table_dir/ | head -5

done
total 3914268
-rw-r--r-- 1 root root    869 Dec 10 08:26 1000001.pickle
-rw-r--r-- 1 root root   1205 Dec 10 08:26 1000025.pickle
-rw-r--r-- 1 root root    323 Dec 10 08:26 1000032.pickle
-rw-r--r-- 1 root root     22 Dec 10 08:26 1000034.pickle



# 以上初始化完成，我擁有:
word_entropy_table:

hash map，使用方式：word_entropy_table['一個'] -> return '一個'的entroy值

word_ID_table:

hash map，使用方式：word_ID_table['關羽'] -> return '關羽'對應到的IDList

ID_word_count_table_dir:

在VM上的一個資料夾，使用方式：pickle.load(ID_word_count_table_dir/ID)，會return一個word_count的hash map

In [ ]:
# 答題前使用，下載jieba
!pip install jieba

import jieba
jieba.set_dictionary('/content/gdrive/My Drive/DataMining/dict.txt.big')

# 自動猜C

In [ ]:
import math
import time

class question_word():
  def __init__(self, word):
    self.word = word
    if word in word_entropy_table:
      self.entropy = word_entropy_table[word]
    else:
      self.entropy = 0

    if word in word_ID_table:
      self.ID_list = word_ID_table[word]
    else:
      self.ID_list = []

  def __eq__(self, other):
    return self.word == other
  def __repr__(self):
    return self.word
  def __hash__(self):
    return hash(self.word)
  def __lt__(self, other):
    return self.entropy < other
  def __gt__(self, other):
    return self.entropy > other

def get_chosen_words(question):
  question_words = set(jieba.cut(question, cut_all=False))
  chosen_words = []

  # 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
  for Q_word in question_words:
    qw = question_word(Q_word)
    if qw.entropy != 0:
      chosen_words.append(question_word(Q_word))

  # 只取題目entropy前 30 %的字，不包含0
  # 經測試覺得前30%應該是最好的數值
  chosen_words = sorted(chosen_words, reverse=True)[:int(len(chosen_words)*0.3)]

  # 取題目entropy中間 80 %的字，不包含0
  #chosen_words = sorted(chosen_words, reverse=True)[int(len(chosen_words)*0.1):int(len(chosen_words)*0.9)]

  return chosen_words

def get_chosen_words_second(question):
  question_words = set(jieba.cut(question, cut_all=False))
  chosen_words = []

  # 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
  for Q_word in question_words:
    qw = question_word(Q_word)
    if qw.entropy != 0:
      chosen_words.append(question_word(Q_word))

  return chosen_words

"""
def get_options(options):
  for i in range(len(options)):
    options[i] = list(jieba.cut(options[i], cut_all=False))
    for j in range(len(options[i])):
      options[i][j] = question_word(options[i][j])
  return options
"""
def get_options(options):
  for i in range(len(options)):
    options[i] = [question_word(options[i])]
  return options

def union_length(lst1, lst2): 
  # return the length of union between lst1,lst2
  return len(set(lst1) & set(lst2)) 

def get_union(lst1, lst2): 
  # return the length of union between lst1,lst2
  return list(set(lst1) & set(lst2)) 

def get_grades(chosen_words, options):
  scores = []
  for opt in options:
    # 這個option的score
    score = 0

    # opt 為 ['國立', '高雄', '大學']這種形式

    for opt_word in opt:
      # opt_word 為'國立'這種單詞
      if opt_word in word_ID_table:
        opt_word_ID_list = word_ID_table[opt_word]

        # 把問題被選擇的那幾個字(最多5個)挑出來
        for QW in chosen_words:
          ID_count = union_length(opt_word_ID_list, QW.ID_list)
          ID_union = get_union(opt_word_ID_list, QW.ID_list)
          # 把選項對應到的ID_list挑出來
          total_ID_word_count = 0
          for ID in ID_union:
            with open('ID_word_count_table_dir/' + str(ID) + '.pickle', 'rb') as f:
              word_count_table = pickle.load(f)
            f.close()
            try:
              total_ID_word_count += word_count_table[QW]
            except:
              _ = 1

          # 以下選一種評分方式

          #if ID_count:
          #  score += math.log(ID_count, 2)

          #score += QW.entropy * total_ID_word_count
          
          if total_ID_word_count:
            score += math.log(total_ID_word_count)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1) * max(math.log(ID_count, 2), 1)

    scores.append(score)
  return scores

import json
#with open('/content/gdrive/My Drive/DataMining/Questions_with_Ans.json') as f:
#  json_questions = json.load(f)
with open('/content/gdrive/My Drive/DataMining/14_Question.json') as f:
  json_questions = json.load(f)

print("start")
tStart = time.time()

print("{", end = '')

est_answers = []
count = 0
for q in json_questions:
  question = q['Question']
  options = [q['A'], q['B'], q['C']]

  chosen_words = get_chosen_words(question)
  options = get_options(options)

  grades = get_grades(chosen_words, options)
  if grades[0] == grades[1] and grades[1] == grades[2]:
    # 假如三個答案分數相等，那麼取全部的詞再評分一次
    chosen_words = get_chosen_words_second(question)
    grades = get_grades(chosen_words, options)

  if grades[0] > grades[1] and grades[0] > grades[2]:
    print("A", end = ",")

    """
    if q['Answer'] != "A":
      #print(question)
      #print(chosen_words)
      #print("\"", q['A'], "\"", q['B'], q['C'])
      #print(grades)
      #print()
      count += 1
    """

    est_answers.append("A")
  elif grades[1] > grades[0] and grades[1] > grades[2]:
    print("B", end = ",")

    """
    if q['Answer'] != "B":
      #print(question)
      #print(chosen_words)
      #print(q['A'], "\"", q['B'], "\"", q['C'])
      #print(grades)
      #print()
      count += 1
    """

    est_answers.append("B")
  else:
    print("C", end = ",")

    """
    if q['Answer'] != "C":
      #print(question)
      #print(chosen_words)
      #print(q['A'], q['B'], "\"", q['C'], "\"")
      #print(grades)
      #print()
      count += 1
    """

    est_answers.append("C")
  #print(grades)
print("}")

tEnd = time.time()
print("It cost %f sec"%(tEnd - tStart))
print("Grades:", (200-count)*0.5)
print()

print("!!!!!!!!!!!!!please copy the output below as the answer!!!!!!!!!!!!!")
print('["' + est_answers[0] + '"' , end = '')
for ans in est_answers[1:]:
  print(', "' + ans + '"', end = '')
print(']')


start
{A,A,B,C,B,B,A,A,C,A,C,C,B,A,A,C,C,A,A,C,C,B,C,B,C,C,A,C,A,C,B,C,A,C,B,A,A,B,C,C,B,C,C,B,B,B,A,A,A,C,A,C,C,C,A,C,A,A,C,A,A,C,A,A,A,B,C,A,C,B,C,C,B,B,A,A,B,A,B,A,B,C,B,C,A,C,A,B,B,A,A,C,B,A,C,A,C,C,A,B,B,C,A,B,C,B,A,B,B,C,C,C,B,C,B,B,A,C,C,B,A,A,B,C,A,A,C,B,B,C,C,C,B,A,A,B,B,A,A,C,B,C,C,C,C,A,C,B,A,A,B,B,A,C,B,A,B,B,C,B,A,B,B,C,A,A,C,C,B,B,B,A,B,B,A,B,A,C,C,A,B,C,C,C,C,B,A,B,C,B,B,B,B,A,C,C,A,C,C,A,}
It cost 11.267268 sec
Grades: 100.0

!!!!!!!!!!!!!please copy the output below as the answer!!!!!!!!!!!!!
["A", "A", "B", "C", "B", "B", "A", "A", "C", "A", "C", "C", "B", "A", "A", "C", "C", "A", "A", "C", "C", "B", "C", "B", "C", "C", "A", "C", "A", "C", "B", "C", "A", "C", "B", "A", "A", "B", "C", "C", "B", "C", "C", "B", "B", "B", "A", "A", "A", "C", "A", "C", "C", "C", "A", "C", "A", "A", "C", "A", "A", "C", "A", "A", "A", "B", "C", "A", "C", "B", "C", "C", "B", "B", "A", "A", "B", "A", "B", "A", "B", "C", "B", "C", "A", "C", "A", "B", "B", "A", "A", "C", "B", "A", "C", "A", "C",

# 手動填寫答案

In [ ]:
import math
import time

class question_word():
  def __init__(self, word):
    self.word = word
    if word in word_entropy_table:
      self.entropy = word_entropy_table[word]
    else:
      self.entropy = 0

    if word in word_ID_table:
      self.ID_list = word_ID_table[word]
    else:
      self.ID_list = []

  def __eq__(self, other):
    return self.word == other
  def __repr__(self):
    return self.word
  def __hash__(self):
    return hash(self.word)
  def __lt__(self, other):
    return self.entropy < other
  def __gt__(self, other):
    return self.entropy > other

def get_chosen_words(question):
  question_words = set(jieba.cut(question, cut_all=False))
  chosen_words = []

  # 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
  for Q_word in question_words:
    qw = question_word(Q_word)
    if qw.entropy != 0:
      chosen_words.append(question_word(Q_word))

  # 只取題目entropy前 30 %的字，不包含0
  # 經測試覺得前30%應該是最好的數值
  chosen_words = sorted(chosen_words, reverse=True)[:int(len(chosen_words)*0.3)]

  # 取題目entropy中間 80 %的字，不包含0
  #chosen_words = sorted(chosen_words, reverse=True)[int(len(chosen_words)*0.1):int(len(chosen_words)*0.9)]

  return chosen_words

def get_chosen_words_second(question):
  question_words = set(jieba.cut(question, cut_all=False))
  chosen_words = []

  # 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
  for Q_word in question_words:
    qw = question_word(Q_word)
    if qw.entropy != 0:
      chosen_words.append(question_word(Q_word))

  return chosen_words

"""
def get_options(options):
  for i in range(len(options)):
    options[i] = list(jieba.cut(options[i], cut_all=False))
    for j in range(len(options[i])):
      options[i][j] = question_word(options[i][j])
  return options
"""
def get_options(options):
  for i in range(len(options)):
    options[i] = [question_word(options[i])]
  return options

def union_length(lst1, lst2): 
  # return the length of union between lst1,lst2
  return len(set(lst1) & set(lst2)) 

def get_union(lst1, lst2): 
  # return the length of union between lst1,lst2
  return list(set(lst1) & set(lst2)) 

def get_grades(chosen_words, options):
  scores = []
  for opt in options:
    # 這個option的score
    score = 0

    # opt 為 ['國立', '高雄', '大學']這種形式

    for opt_word in opt:
      # opt_word 為'國立'這種單詞
      if opt_word in word_ID_table:
        opt_word_ID_list = word_ID_table[opt_word]

        # 把問題被選擇的那幾個字(最多5個)挑出來
        for QW in chosen_words:
          ID_count = union_length(opt_word_ID_list, QW.ID_list)
          ID_union = get_union(opt_word_ID_list, QW.ID_list)
          # 把選項對應到的ID_list挑出來
          total_ID_word_count = 0
          for ID in ID_union:
            with open('ID_word_count_table_dir/' + str(ID) + '.pickle', 'rb') as f:
              word_count_table = pickle.load(f)
            f.close()
            try:
              total_ID_word_count += word_count_table[QW]
            except:
              _ = 1

          # 以下選一種評分方式

          #if ID_count:
          #  score += math.log(ID_count, 2)

          #score += QW.entropy * total_ID_word_count
          
          if total_ID_word_count:
            score += math.log(total_ID_word_count)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1) * max(math.log(ID_count, 2), 1)

    scores.append(score)
  return scores

import json
#with open('/content/gdrive/My Drive/DataMining/Questions_with_Ans.json') as f:
#  json_questions = json.load(f)
with open('/content/gdrive/My Drive/DataMining/14_Question.json') as f:
  json_questions = json.load(f)

print("start")
tStart = time.time()

print("{", end = '')

est_answers = []
act_answers = []
for q in json_questions:
  #act_answers.append(q['Answer'])

  question = q['Question']
  options = [q['A'], q['B'], q['C']]

  chosen_words = get_chosen_words(question)
  options = get_options(options)

  grades = get_grades(chosen_words, options)
  if grades[0] == grades[1] and grades[1] == grades[2]:
    chosen_words = get_chosen_words_second(question)
    grades = get_grades(chosen_words, options)

  if grades[0] == grades[1] and grades[1] == grades[2]:
    print()
    print(question)
    print(options)
    handAns = input()
    while handAns != "A" and handAns != "B" and handAns != "C":
      print("error")
      print("key:", handAns)
      handAns = input()
    print(handAns, end = ",")
    est_answers.append(handAns)
  else:
    if grades[0] > grades[1] and grades[0] > grades[2]:
      print("A", end = ",")

      """
      if q['Answer'] != "A":
        print(question)
        print(chosen_words)
        print("\"", q['A'], "\"", q['B'], q['C'])
        print(grades)
        print()
        count += 1
      """

      est_answers.append("A")
    elif grades[1] > grades[0] and grades[1] > grades[2]:
      print("B", end = ",")

      """
      if q['Answer'] != "B":
        print(question)
        print(chosen_words)
        print(q['A'], "\"", q['B'], "\"", q['C'])
        print(grades)
        print()
        count += 1
      """

      est_answers.append("B")
    else:
      print("C", end = ",")

      """
      if q['Answer'] != "C":
        print(question)
        print(chosen_words)
        print(q['A'], q['B'], "\"", q['C'], "\"")
        print(grades)
        print()
        count += 1
      """

      est_answers.append("C")
  #print(grades)
print("}")

tEnd = time.time()
print("It cost %f sec"%(tEnd - tStart))

"""
count = 0
for i in range(200):
  if est_answers[i] == act_answers[i]:
    count += 1
print("Grades:", count*0.5)
print()
"""

print("!!!!!!!!!!!!!please copy the output below as the answer!!!!!!!!!!!!!")
print('["' + est_answers[0] + '"' , end = '')
for ans in est_answers[1:]:
  print(', "' + ans + '"', end = '')
print(']')


start
{A,A,B,
主角名為哈克,在雪山迷路，記憶盡失、遇到襲擊時被久遠所救的青年。本作的主人公。年齡約二十餘歲。與其他有著獸耳、獸尾的亞人相對、哈克是個人類。因為連自己的名字也想不起、久遠給了他「ハク」這個名字，這個名字是從久遠故鄉傳說「被傳頌之人」的名字而來的。與周圍相比體能極低、連小孩能簡單做到的事也能讓他苦戰，再加上是懶惰者、用著各種各樣的藉口想在工作上偷懶。但是相對洞察力很優秀、頭腦也轉得很快、能下達大膽的作戰與正確的判斷而多次絕處逢生。是一款由日本AQUAPLUS公司在2015年9月24日發售的冒險+戰略角色扮演遊戲
[[揭穿你的謊言], [假面的誘惑], [受讚頌者 虛偽的假面]]
C
C,B,B,A,A,C,A,
哪一個手機於世界各地連番引發爆炸事故，最終決定作全球回收並停止生産。而後調查結果爆炸原因在於電池與組件之間過於緊密所造成?
[[Galaxy Note 7], [iPhone XS], [ZenFone 4]]
A
A,C,B,A,A,C,C,A,A,
是由三座位於北美洲五大湖區尼加拉河上瀑布的總稱，平均流量為2,407立方公尺/秒，與伊瓜蘇瀑布、維多利亞瀑布並稱為世界三大跨國瀑布。以美麗的景色，巨大的水力發電能力和極具挑戰性的環境保護工程而聞名於世，是非常受遊客歡迎的旅遊景點。 整個瀑布跨越加拿大的安大略省和美國的紐約州構成南部的尼亞加拉峽谷。三座瀑布按規模從大到小分別為馬蹄瀑布，美國瀑布，布里達爾維爾瀑布。其中，馬蹄瀑布位於美國和加拿大的邊境，美國瀑布全部位於美國境內並由山羊島分隔開。小些的布里達爾維爾瀑布也在美國境內，由月神島從美國瀑布隔開。
[[翡翠瀑布], [尼加拉瀑布], [印尼拉瀑布]]
B
B,C,B,C,B,C,
哪部是以台灣高雄捷運公司的虛擬代言人做為題材的輕小說系列，台灣由尖端出版社出版、日本由GA文庫出版。
[[前進吧！！北捷少男], [前進吧！！中捷少爺], [前進吧！！高捷少女]]
C
C,A,
由特雷·帕克和馬特·斯通為美國喜劇中心創作的成人動畫情景喜劇劇集稱為?
[[南方四賤客], [北方四賤客], [西方四賤客]]
A
A,A,C,B,C,A,
是一部於2014年上映的美國超級英雄電影，美國隊長的續集，由安東尼及喬·羅素共同執導。
[[無限之戰], [奧創紀元], [酷寒戰士]]
C
C,B,A,A

# 以下內容為測試時使用

In [ ]:
# 測試用
class question_word():
  def __init__(self, word):
    self.word = word
    if word in word_entropy_table:
      self.entropy = word_entropy_table[word]
    else:
      self.entropy = 0

    if word in word_ID_table:
      self.ID_list = word_ID_table[word]
    else:
      self.ID_list = []

  def __eq__(self, other):
    return self.word == other
  def __repr__(self):
    return self.word
  def __hash__(self):
    return hash(self.word)
  def __lt__(self, other):
    return self.entropy < other
  def __gt__(self, other):
    return self.entropy > other

question = "西非國家，位於非洲的幾內亞灣西岸頂點，鄰國包括西邊的貝南，北邊的尼日，東北方與查德接壤一小段國界，正東則是喀麥隆。是全非洲人口最多的國家，首都原本為西南沿海的海港城市拉哥斯，1991年12月遷都至地理位置位居全國國土正中央的阿布札。"
question_words = set(jieba.cut(question, cut_all=False))
chosen_words = []

# 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
for Q_word in question_words:
  qw = question_word(Q_word)
  if qw.entropy != 0:
    chosen_words.append(question_word(Q_word))
# 假如題目的字太多，只選entropy最高的10個來使用
if len(chosen_words) > 10:
  chosen_words = sorted(chosen_words, reverse=True)[:10]

print(chosen_words)

options = ["瑞士", "奈及利亞", "奧地利"]
# 把選項做切字，切完字之後轉成question_word類別
for i in range(len(options)):
  options[i] = list(jieba.cut(options[i], cut_all=False))
  for j in range(len(options[i])):
    options[i][j] = question_word(options[i][j])
print(options)

#question = "簡稱 臺大 ， 前身 為 日治 時期 1928 年 創立 的 臺北 帝國 大學 ， 由 首任 校長 幣原 坦 籌設 ， 原初 只 定位 為 醫學 和 農學 的 實業 大學 ， 由 時任 總督 伊澤多 喜 男 改為 籌設 成 綜合大學 之 目標 。 臺北 帝國 大學 在 二次大戰 後 二次 改名 ， 1945 年 11 月 易名 為 國立 臺北 大學 ， 同年 12 月 15 日 啟用 現名 國立 臺灣大學 ， 以 自由主義 學風 著稱 ， 是 臺灣 第一所 現代 綜合大學"
#option = ('國立高雄大學', '以色列', '孟加拉')

[的, 拉哥斯, 尼日, 貝南, 西非國家, 正東, 幾內亞灣, 查德, 1991, 國界]
[[瑞士], [奈及利亞], [奧地利]]


In [ ]:
import time
tStart = time.time()

scores = []
for opt in options:
  print(opt)
  # 這個option的score
  score = 0

  # opt 為 ['國立', '高雄', '大學']這種形式

  for opt_word in opt:
    # opt_word 為'國立'這種單詞
    if opt_word in word_ID_table:
      opt_word_ID_list = word_ID_table[opt_word]

      # 把問題被選擇的那幾個字(最多5個)挑出來
      for QW in chosen_words:
        ID_count = 0
        # 把選項對應到的ID_list挑出來
        for ID in opt_word_ID_list:
          if ID in QW.ID_list:
            ID_count += 1
        # 分數加上問題的字的entropy * 這個選項對應到這個字的ID match次數
        score += QW.entropy * ID_count
  scores.append(score)

tEnd = time.time()
print("It cost %f sec"%(tEnd - tStart))

print(scores)

[瑞士]
[奈及利亞]
[奧地利]
It cost 0.101728 sec
[316.8610860674841, 530.5113459129278, 371.8522737823538]


In [ ]:
# 測試用

!pip install jieba
!wget https://www.dropbox.com/s/ikv3n0fzb218vgn/dict.txt.big?dl=1

!mv dict.txt.big?dl=1 dict.txt.big
!ls

--2020-11-02 12:50:01--  https://www.dropbox.com/s/ikv3n0fzb218vgn/dict.txt.big?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/ikv3n0fzb218vgn/dict.txt.big [following]
--2020-11-02 12:50:02--  https://www.dropbox.com/s/dl/ikv3n0fzb218vgn/dict.txt.big
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8ffd1a438e5d3d2d325af580fb.dl.dropboxusercontent.com/cd/0/get/BCZGrftUb060G06na43LYNTBeVB8AVMvt1bc6oZfF-q_cWKrMBA_LpPfOTHNHNyWdihd2q-FvL3M8i8MZL1p3zRgjiP4h7k6sHSBWmvddWmjhQ/file?dl=1# [following]
--2020-11-02 12:50:02--  https://uc8ffd1a438e5d3d2d325af580fb.dl.dropboxusercontent.com/cd/0/get/BCZGrftUb060G06na43LYNTBeVB8AVMvt1bc6oZfF-q_cWKrMBA_LpPfOTHNHNyWdihd2q-FvL3M8i8MZL1p3zRgjiP4h7k6sHSBWmvddWmjhQ/file?dl=1
Resolving

In [ ]:
import jieba
jieba.set_dictionary('dict.txt.big')

sentence = "國立臺灣大學"
words = jieba.cut(sentence, cut_all=False)

for word in words:
    print(word, end = ' ')

Building prefix dict from /content/dict.txt.big ...
Loading model from cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.541 seconds.
Prefix dict has been built successfully.


國立 臺灣大學 

In [ ]:
import math
import time

class question_word():
  def __init__(self, word):
    self.word = word
    if word in word_entropy_table:
      self.entropy = word_entropy_table[word]
    else:
      self.entropy = 0

    if word in word_ID_table:
      self.ID_list = word_ID_table[word]
    else:
      self.ID_list = []

  def __eq__(self, other):
    return self.word == other
  def __repr__(self):
    return self.word
  def __hash__(self):
    return hash(self.word)
  def __lt__(self, other):
    return self.entropy < other
  def __gt__(self, other):
    return self.entropy > other

def get_chosen_words(question):
  question_words = set(jieba.cut(question, cut_all=False))
  chosen_words = []

  # 將題目的字轉成question_word類別，只有有rntropy的詞才會加入
  for Q_word in question_words:
    qw = question_word(Q_word)
    if qw.entropy != 0:
      chosen_words.append(question_word(Q_word))

  # 只取題目entropy前 30 %的字，不包含0
  # 經測試覺得前30%應該是最好的數值
  chosen_words = sorted(chosen_words, reverse=True)[:int(len(chosen_words)*0.3)]

  # 取題目entropy中間 80 %的字，不包含0
  #chosen_words = sorted(chosen_words, reverse=True)[int(len(chosen_words)*0.2):int(len(chosen_words)*0.8)]

  return chosen_words

"""
def get_options(options):
  for i in range(len(options)):
    options[i] = list(jieba.cut(options[i], cut_all=False))
    for j in range(len(options[i])):
      options[i][j] = question_word(options[i][j])
  return options
"""
def get_options(options):
  for i in range(len(options)):
    options[i] = [question_word(options[i])]
  return options

def union_length(lst1, lst2): 
  # return the length of union between lst1,lst2
  return len(set(lst1) & set(lst2)) 

def get_union(lst1, lst2): 
  # return the length of union between lst1,lst2
  return list(set(lst1) & set(lst2)) 

def get_grades(chosen_words, options):
  scores = []
  for opt in options:
    # 這個option的score
    score = 0

    # opt 為 ['國立', '高雄', '大學']這種形式

    for opt_word in opt:
      # opt_word 為'國立'這種單詞
      if opt_word in word_ID_table:
        opt_word_ID_list = word_ID_table[opt_word]

        # 把問題被選擇的那幾個字(最多5個)挑出來
        for QW in chosen_words:
          ID_count = union_length(opt_word_ID_list, QW.ID_list)
          ID_union = get_union(opt_word_ID_list, QW.ID_list)
          # 把選項對應到的ID_list挑出來
          total_ID_word_count = 0
          for ID in ID_union:
            with open('ID_word_count_table_dir/' + str(ID) + '.pickle', 'rb') as f:
              word_count_table = pickle.load(f)
            f.close()
            try:
              total_ID_word_count += word_count_table[QW]
            except:
              a = 1

          # 以下選一種評分方式

          #if ID_count:
          #  score += math.log(ID_count, 2)

          #score += QW.entropy * total_ID_word_count
          
          if total_ID_word_count:
            score += math.log(total_ID_word_count)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1)

          #if (total_ID_word_count):
          #  score += max(math.log(total_ID_word_count, 2), 1) * max(math.log(ID_count, 2), 1)

    scores.append(score)
  return scores

import json
with open('/content/gdrive/My Drive/DataMining/Questions_with_Ans.json') as f:
  json_questions = json.load(f)

print("start")
tStart = time.time()

print("{", end = '')

est_answers = []
answers = []
for q in json_questions:
  question = q['Question']
  options = [q['A'], q['B'], q['C']]
  answers.append(q['Answer'])

  chosen_words = get_chosen_words(question)
  options = get_options(options)

  grades = get_grades(chosen_words, options)

  if grades[0] > grades[1] and grades[0] > grades[2]:
    print("A", end = ",")
    est_answers.append("A")
  elif grades[1] > grades[0] and grades[1] > grades[2]:
    print("B", end = ",")
    est_answers.append("B")
  else:
    print("C", end = ",")
    est_answers.append("C")
print("}")

tEnd = time.time()
print("It cost %f sec"%(tEnd - tStart))

final_score = 0
for i in range(len(answers)):
  if (answers[i] == est_answers[i]):
    final_score += 1
print(final_score)

print()

print("!!!!!!!!!!!!!please copy the output below as the answer!!!!!!!!!!!!!")
print('["' + est_answers[0] + '"' , end = '')
for ans in est_answers[1:]:
  print(', "' + ans + '"', end = '')
print(']')


start
{B,C,C,A,C,C,C,C,C,B,C,C,B,C,A,C,A,C,B,C,C,A,C,C,A,C,B,C,C,C,C,C,C,C,A,B,C,C,B,A,B,C,A,C,A,C,A,B,A,C,B,B,C,C,C,B,B,B,C,B,C,B,C,C,C,A,B,C,C,C,C,B,C,C,C,C,B,C,C,C,A,C,B,A,C,C,C,C,C,C,B,C,A,A,C,A,C,C,B,C,C,C,B,B,C,A,C,C,C,A,C,B,C,B,C,C,A,C,B,C,B,C,A,C,C,A,B,C,B,A,A,B,B,C,C,C,A,B,C,A,A,C,A,A,B,C,B,A,A,C,A,A,B,B,B,A,A,C,C,A,A,B,A,C,A,A,B,A,C,C,C,C,A,C,A,C,B,C,A,B,A,C,A,A,C,A,A,A,B,B,B,C,C,A,A,B,B,B,B,B,}
It cost 2.848435 sec
148

!!!!!!!!!!!!!please copy the output below as the answer!!!!!!!!!!!!!
["B", "C", "C", "A", "C", "C", "C", "C", "C", "B", "C", "C", "B", "C", "A", "C", "A", "C", "B", "C", "C", "A", "C", "C", "A", "C", "B", "C", "C", "C", "C", "C", "C", "C", "A", "B", "C", "C", "B", "A", "B", "C", "A", "C", "A", "C", "A", "B", "A", "C", "B", "B", "C", "C", "C", "B", "B", "B", "C", "B", "C", "B", "C", "C", "C", "A", "B", "C", "C", "C", "C", "B", "C", "C", "C", "C", "B", "C", "C", "C", "A", "C", "B", "A", "C", "C", "C", "C", "C", "C", "B", "C", "A", "A", "C", "A", "C", "C", "B", 